# Performing analysis with GeoPandas
With spatial data stored as a geodataframe, we can run a number of analyses, both tabular, using Pandas operations, and spatial, using Geopandas operations. This notebook wades gently into the world of GeoPandas and also serves as a review of several Pandas operations. 

Specifically we examine the following:
* Reading & writing shapefiles into a GeoPandas dataFrame
* Reprojecting data using GeoPandas
* Exploring the GeoPandas dataFrame
 * Exposing the number of features 
 * Revealing the data types of each column 
 * Exploring the `geometry` data type
* Quick view of plotting in GeoPandas
* Quick view of geoprocessing in GeoPandas

### Install the package and enable inline plots

In [ ]:
#import the package
import geopandas as gpd

#enable plots to appear in the notebook
%matplotlib inline

---
<h2><font color='red'>► FIX REQUIRED ◄</font></h2>
*You need to run this code block to fix an issue with the `pyproj` module used by GeoPandas.*

For more on the issue see: 
https://github.com/geopandas/geopandas/issues/830

In [ ]:
#Fix issue with pyproj 
import sys, os
pythonPath = sys.executable
pythonFolder = os.path.dirname(pythonPath)
shareFolder = os.path.join(pythonFolder,'Library','share')
os.environ["PROJ_LIB"] = shareFolder

---
### Read a shapefile into a _GeoDataframe_
GeoPandas can read shapefiles directly. Behind the scenes, this operation is using the `GDAL` package which contains the binaries capable of understanding geospatial data, the `fiona` package, which allows Python to interact nicely with `GDAL` libraries, and the `shapely` package which has functions for operating with feature classes in a Pythonic way. GeoPandas coordinate reference systems can use the "European Petroleum Survey Group" (EPSG) codes as shorthand for various standard systems. 

Complete documentation on the GeoDataframe is here: http://geopandas.org/data_structures.html#geodataframe

In [ ]:
#read in the HUC12.shp feature class
gdf = gpd.read_file('./Data/HUC12.shp')

In [ ]:
#How many features in the dataset?
len(gdf)

In [ ]:
#What coordinate reference system is used? Check http://epsg.io for what this code is
gdf.crs

→ If the crs returns an 'epsg' code you can generate a URL to look it up...

In [ ]:
#Get the epsg code from the crs
epsg = gdf.crs['init'].split(':')[1]
#Generate and print the URL, which you can click on...
print("http://epsg.io/{}".format(epsg))

In [ ]:
#show the geometry types in this geodataframe
gdf.type.unique()

In [ ]:
#examine the attributes for the first feature
gdf.iloc[0]

### Projections in GeoPandas: Reprojecting data
We can reproject our data in GeoPandas. Here we'll reproject our NAD83 data to UTM Zone 17, which has an EPSG code of `26917`. 

In [ ]:
gdfUTM  = gdf.to_crs({'init':'epsg:26917'})

### Exploring the `geometry` objects in a GeoPandas dataframe
The key to Geopandas ability to work with geospatial data is by adding a new data type to the standard Pandas dataFrame: this is stored in the `geometry` field. Let's explore this field. 

Complete documentation on the geometry object is here: http://geopandas.org/geometric_manipulations.html

In [ ]:
#show the first 5 values in the geometry field: this is actually a GeoSeries...
gdfUTM['geometry'][0:5]

In [ ]:
#show just the first value - it appears as a shape
gdfUTM['geometry'][0]

Now let's save that one geometry object, a polygon in this case to variable and examine what GeoPandas allows us to do with it. 

In [ ]:
#Extract one feature geometry to a variable; what is its datatype?
thePoly = gdfUTM['geometry'][0]
type(thePoly)

In [ ]:
#Show thePoly
thePoly

In [ ]:
#Show the area and perimeter length
theArea = thePoly.area
thePerim = thePoly.length
print ("Area (m2):",int(theArea))
print ("Permeter (m):",int(thePerim))

In [ ]:
#Convert the polygon's boundary to a linestring (i.e. a line feature)
theBoundary = thePoly.boundary
type(theBoundary)

In [ ]:
#Show the 
theBoundary

In [ ]:
#Create the centroid of the feature
theCentroid = thePoly.centroid
type(theCentroid)

In [ ]:
theCentroid

In [ ]:
#Buffer the polygon 100m
thePoly.buffer(100)

→ Other feature transformations to try: `convex_hull`, `envelope`, `simplify`...

#### Distance analyses
We can also compute distances fairly easily with GeoPandas objects. Here we'll compute the distance of each feature to the center point of all features.

In [ ]:
#Compute the center of all features combined
theCenter = gdfUTM.unary_union.centroid #distance(durhamPtDD)

In [ ]:
#Compute the distance of each feature to this center point & show the mean
theDistances = gdfUTM.distance(theCenter)/1000
theDistances.mean()

In [ ]:
#Plot a histogram of values
theDistances.hist(figsize=(15,3));

### A more complex analyis
Here we will buffer the centroid of a feature and then intersect that with the feature. 

* We begin by selecting a feature. We'll pick on the Elk Creek HUC...

In [ ]:
#Select a feature by an attribute
hucMask = gdfUTM['HU_12_NAME'] == 'Elk Creek'
gdfHUC = gdfUTM[hucMask]
type(gdfHUC)

In [ ]:
gdfHUC

►This approach is slightly different than in the above example (`thePoly = gdfUTM['geometry'][0]`) which returned a _Shapely geometry_ object from the geodataframe; here our query returns a _GeoSeries_ object. However, other than plotting, the behavior is mostly the same.

In [ ]:
#Get the shape of the feature
feature_geometry = gdfHUC['geometry'] #->returns a GeoSeries, not a shapely geometry
type(feature_geometry)

In [ ]:
#Copy the dfHUC dataframe and then we'll modify geometries
gdfHUC_copy = gdfHUC.copy(deep=True)

In [ ]:
#Update geometry to the centroid of each feature buffered 5000m
gdfHUC_copy['geometry'] = gdfHUC_copy['geometry'].centroid.buffer(5000)

In [ ]:
#Buffer the centroid
theBuffer = theCentroid.buffer(100)
#Intersect the buffer and the original shape
theClip = gpd.overlay(gdfHUC_copy,gdfHUC,how='intersection')
#Show the Clip
theClip.plot()

## Geospatial capabilities of the GeoPandas dataFrame object

In [ ]:
#Dissolving
dfHUC8 = gdf.dissolve(by='HUC_8',aggfunc='sum')
dfHUC8.dtypes

In [ ]:
dfHUC8.plot(column='ACRES',
            scheme='quantiles',        
            figsize=(14,18));

## Recap
In this super quick introduction to GeoPandas, we saw that the GeoDataFrame is easy to construct from a shapefile, and once constructed gives us access to the analytic capability of Pandas dataframes (e.g. selecting, summarizing, etc.) as well as plotting and spatial analytic capability. 

I'm hopeful that at the end of this short introduction you're eager to read up on the documentation and learn more what GeoPandas can do. 